## Authentication to Spotify API

### Imports

In [20]:
import pandas as pd
import requests
import base64
import json
import csv
import os

### Request for access token

In [21]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [22]:
auth_string = client_id + ":" + client_secret
auth_bytes = auth_string.encode("utf-8")
auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

In [23]:
url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": "Basic " + auth_base64,
    "Content-type": "application/x-www-form-urlencoded"
}
data = {"grant_type": "client_credentials"}

In [24]:
result = requests.post(url, headers=headers, data=data)
result

<Response [200]>

In [25]:
json_result = json.loads(result.content)
token = json_result["access_token"]
token

'BQAfGu3V8Y4Dpxv6ErafnXM1YHLPfRU6hSObQqmuMxRPB6ejnXtwZEZ48DEwJC9nr3R-oD73aLcBo2POkc-dzXbwcBi8zTBsRVbRZecA6--ZjXZzQLY'

In [26]:
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}
# get_auth_header(token)

### Searching for artist

In [27]:
def search_artist(token, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=2"
    
    query_url = search_url + query
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)
    
    return json_result


In [28]:
# search_artist(token, "Kanye West")

### Getting the id

In [29]:
def get_artist_id(token, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=2"
    
    query_url = search_url + query
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    
    artist_id = json_result[0]
    
    if len(json_result) == 0:
        return f"No artists with this name"
    
    return artist_id["id"]

In [30]:
get_artist_id(token, "Kanye West")

'5K4W6rqBFWDnAN6FQUkS6x'

### Getting the artist name

In [31]:
def get_artist_name(token, id):
    search_url = f"https://api.spotify.com/v1/artists/{id}"
    headers = get_auth_header(token)
    
    result = requests.get(search_url, headers=headers)
    json_result = json.loads(result.content)["name"]
    
    return json_result

In [13]:
test = get_artist_name(token, "5K4W6rqBFWDnAN6FQUkS6x")
test

'Kanye West'

### Getting the related artists

In [32]:
def get_related_artists(token, id):
    artists_url = f"https://api.spotify.com/v1/artists/{id}/related-artists"
    headers = get_auth_header(token)
    
    result = requests.get(artists_url, headers=headers)
    json_result = json.loads(result.content)["artists"]
    
    return json_result

In [15]:
artists = get_related_artists(token, "3nFkdlSjzX9mRTtwJOzDYB")
# artists
# len(artists)

### Automating the process

In [28]:
related_artist = {}
related_artist["main_artist"] = []
related_artist["id"] = []
related_artist["name"] = []
related_artist["popularity"] = []
related_artist["genres"] = []

In [33]:
def collect_related_artists(token, artist_name):
    related_artist["main_artist"].append(artist_name)
    main_artist_id = get_artist_id(token, artist_name)
    artists = get_related_artists(token, main_artist_id)
    
    for i in range(len(artists)):
        related_artist["id"].append(artists[i]["id"])
        related_artist["name"].append(artists[i]["name"])
        related_artist["popularity"].append(artists[i]["popularity"])
        related_artist["genres"].append(artists[i]["genres"])
    
    data_trash = related_artist.copy()
    aux = 0
    while (len(data_trash["id"]) != 0):
        related_artist["main_artist"].append(get_artist_name(token, artists[aux]["id"]))
        # print(aux)
        side_artists = get_related_artists(token, artists[aux]["id"])
        
        for i in range(len(side_artists)):
            related_artist["id"].append(side_artists[i]["id"])
            related_artist["name"].append(side_artists[i]["name"])
            related_artist["popularity"].append(side_artists[i]["popularity"])
            related_artist["genres"].append(side_artists[i]["genres"])
        
        data_trash["id"].pop(0)
        aux += 1
        if aux == 20:
            return related_artist
    
        

In [13]:
# data_rap = collect_related_artists(token, "Ludacris")
# data_pop = collect_related_artists(token, "Steven Tyler")

In [14]:
# data_pop

In [88]:
def generating_csv(file_name, dictionary):
    with open(f"../data/{file_name}.csv", mode="w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        
        keys = dictionary.keys()
        
        csv_writer.writerow(keys)
        
        for i in range(len(dictionary["main_artist"])):
            line = [dictionary[key][i] for key in keys]
            csv_writer.writerow(line)
        
        return "ok"

In [90]:
# generating_csv("pop_artist", data_pop)

'ok'

## Fluxo
### Passo 1
Importar o CSV
### Passo 2
Criar um novo CSV contendo os dados antigos mais uma coluna de relação
### Passo 3
Iterar o CSV antigo pra cada user, obter os dados antigos + a relação 
### Passo 4
Após tudo, escrever a linha

In [69]:

def generating_data(file_name):
    initial_data = pd.read_csv("../data/rappers.csv", encoding='utf-8')
    
    with open(f"../data/{file_name}.csv", mode="w", encoding='utf-8', newline="") as csv_file:
        writer = csv.writer(csv_file)
        
        headers = ["main_artist", "id", "name", "popularity", "genres", "relations"]
        
        writer.writerow(headers)
        
        for artist in initial_data:
            print(initial_data)
            # print(artist)
            relations = get_related_artists(token, get_artist_id(token, "Kanye West"))
            artists_fields = [
                artist.main_artist,
                artist.id,
                artist.name,
                artist.popularity,
                artist.genres,
                artist.relations,
                ",".join(relations)
            ]
            
            writer.writerow(artists_fields)
        
        print(f"Users information saved to {file_name}")

In [70]:
generating_data("teste")

         main_artist                      id              name  popularity  \
0         Kanye West  6DPYiyq5kWVQS4RGwxzPC7             JAY-Z          81   
1              JAY-Z  3Mcii5XWf6E0lrY3Uky4cA          Kid Cudi          77   
2           Kid Cudi  1Oa0bMld0A3u5OTYfMzp5h    Kendrick Lamar          86   
3     Kendrick Lamar  7B4hKK0S9QYnaoqa9OuwgX        A$AP Rocky          79   
4         A$AP Rocky  4EnEZVjo3w1cwcQYePccay         Baby Keem          75   
...              ...                     ...               ...         ...   
1045  Chamillionaire  5IcR3N7QB1j6KBL8eImZ8m        Gang Starr          61   
1046     Trick Daddy  68kEuyFKyqrdQQLLsmiatm  Ghostface Killah          63   
1047        Mystikal  2hPgGN4uhvXAxiXQBIXOmE        Method Man          67   
1048    David Banner  5Matrg5du62bXwer29cU5T            Common          58   
1049  Jermaine Dupri  0TkqXdyWLsssJH7okthMPQ            Redman          61   

                                                 genres  
0    

AttributeError: 'str' object has no attribute 'main_artist'